In [ ]:
import os
import sys
import numpy as np

# Add the path to the custom library to the system path
sys.path.append('..')

# Import custom modules
from src import Tensor
from src.core.utils import data_analysis, context_manager
from src.architectures.transformer import Tokenizer, Transformer, DataLoader

### Constants and hyperparameters

In [ ]:
# Constants
dataset_path = os.path.join(os.getcwd(), 'dataset', 'divina_commedia.txt')
tokenizer_path = os.path.join(os.getcwd(), 'checkpoints', 'tokenizer.json')
model_path = os.path.join(os.getcwd(), 'checkpoints', 'language_model.npz')

In [ ]:
# Hyperparameters
dropout = 0.2 # The dropout rate
train_val_split = 0.9 # 90% of the data will be used for training, 10% for validation
batch_size = 32 # The number of samples to use for each batch
grad_accumulation_steps = 1 # The number of steps to accumulate gradients before updating the model
sequence_length = 256 # The size of the sequence length (the context window)
learning_rate = 1e-3 # The learning rate for the optimizer
training_steps = 3 # The number of steps to train the model for
n_embed = 384 # The size of the token embeddings (the dimensionality of the embeddings)
eval_iters = 1 # The number of iterations to evaluate the model
n_attention_heads = 6 # The number of attention heads in the multi-head attention mechanism
n_decoder_blocks = 6 # The number of transformer'decoder blocks in the model

### Data loading

In [ ]:
def load_txt_file(path: str) -> str:
    """
    Load a text file from the specified path.
    
    Parameters:
    - path (str): The path to the text file.
    
    Returns:
    - str: The contents of the text file.
    """
    
    # Check if the file exists
    if not os.path.exists(path):
        raise FileNotFoundError(f'The file "{path}" does not exist.')
    
    # Read the file
    with open(path, 'r', encoding='utf-8') as file:
        return file.read()

In [ ]:
# Instantiate the tokenizer
tokenizer = Tokenizer()

# Load the state of the tokenizer
tokenizer.load(tokenizer_path)

# Extract the vocabulary size
vocab_size = tokenizer.get_vocab_size()

In [ ]:
# Load the text file
text = load_txt_file(dataset_path)

# Encode the text using the tokenizer
encoded_text = tokenizer.encode(text)

# Convert the data to a tensor
data = Tensor(np.array(encoded_text), dtype=np.int32)

In [ ]:
# Instantiate the data loader
data_loader = DataLoader(
    data = data, 
    train_val_split = train_val_split
)

### Building the model

In [ ]:
# Create the language model
language_model = Transformer(
    name = "Language Model",
    vocab_size = vocab_size,
    n_embed = n_embed,
    n_attention_heads = n_attention_heads,
    sequence_length = sequence_length,
    n_decoder_blocks = n_decoder_blocks,
    dropout = dropout
)

### Initializing the model

In [ ]:
# Check if the model is already trained
if os.path.exists(model_path):
    # Load the model to continue training
    print("Loading the model from the checkpoint...")
    language_model.load(model_path)
    print("Model loaded successfully.")

In [ ]:
# Call the model with a first batch to initialize the weights
# This is not necessary, but it is useful to know the input size

# Disable gradient computation
with context_manager.no_grad():
    # Set the model in evaluation mode
    language_model.eval()
    
    # Get a batch of data
    x, _ = data_loader.get_batch(
        batch_size = batch_size,
        sequence_length = sequence_length
    )
    
    # Call the model with a batch of data to initialize it
    language_model(x)

In [ ]:
# Display the model summary in tree format.
# This is useful since the whole model is composed of submodules,
# therefore, the model summary will be displayed recursively
language_model.summary(recursive=True)

### Training the model

In [ ]:
# Train the model
history = language_model.fit(
    data_loader = data_loader,
    steps = training_steps,
    lr = learning_rate,
    batch_size = batch_size,
    eval_iters = eval_iters,
    grad_accumulation_steps = grad_accumulation_steps,
)

In [ ]:
# Save the model
language_model.save(model_path)

In [ ]:
# Plot the training and validation loss
data_analysis.plot_history(history["loss"].data, history["val_loss"].data, "Training and Validation Loss", "Eval iter", "Loss")

### Inference

In [ ]:
# Generate some text context from the trained model
context = Tensor(np.zeros((1, 1), dtype=np.int32))

# Iterate over the tokens generated by the transformer
for token in language_model.generate(context, max_new_tokens=200, stream=True):
    # Decode the token
    decoded_token = tokenizer.decode([token.data.squeeze().tolist()])

    # Print the decoded token
    print(decoded_token, end='', flush=True)